In [1]:
import os
import pandas as pd
from datetime import datetime

def retrieve_data(sheet_name):
    """Retrieve data from Excel file based on sheet name."""
    return pd.read_excel("../data/option_data/linh.xlsx", 
                        sheet_name=sheet_name, 
                        header=0, 
                        skiprows=0, 
                        na_values=['NA', 'NaN'])

def compute_maturity(date_str, start_date="2024/07/25", end_date="2024/10/25"):
    """Compute maturity in years from the ticker's expiration date."""
    try:
        start_date = datetime.strptime(start_date, "%Y/%m/%d")
        end_date = datetime.strptime(end_date, "%Y/%m/%d")
        maturity_date = datetime.strptime(date_str, "%m/%d/%y")
        return maturity_date
    except Exception as e:
        print(f"Error parsing date: {e}")
        return None

def standardize_date_format(date_str):
    """Convert various date formats to DDMMYYYY format."""
    # Handle different input formats
    try:
        if len(date_str) == 8:  # Already in DDMMYYYY format
            return date_str
        if len(date_str) == 6:  # DDMMYY format
            return date_str[:4] + "20" + date_str[4:]
        # Add more format handlers if needed
        return date_str
    except Exception as e:
        print(f"Error standardizing date: {e}")
        return date_str

def process_and_save(sheet_name):
    """Process and save data to appropriate directory structure."""
    try:
        # Extract date and category from sheet name
        parts = sheet_name.split('_')
        category = parts[1]  # rog, cfr, or zurn
        date_str = parts[-1]  # The date part
        
        # Standardize date format
        std_date = standardize_date_format(date_str)
        
        # Create base directory for the date
        base_dir = f"../data/option_data/{std_date}"
        os.makedirs(base_dir, exist_ok=True)
        
        # Retrieve and process data
        data = retrieve_data(sheet_name)
        formatted_data = []
        
        for index, row in data.iterrows():
            if pd.notna(row['Ticker']) and row['Ticker'].startswith(category.upper()):
                ticker_parts = row['Ticker'].split(' ')
                if len(ticker_parts) > 1:
                    maturity = compute_maturity(ticker_parts[1])
                    if maturity is not None:
                        # Calculate mid price
                        price = (row['Bid'] + row['Ask']) / 2 if pd.notna(row['Bid']) and pd.notna(row['Ask']) else None
                        
                        formatted_data.append([
                            maturity,
                            int(row['Strike']),
                            price,
                            row['IVM']
                        ])
        
        # Create and save DataFrame
        if formatted_data:
            formatted_df = pd.DataFrame(formatted_data, 
                                      columns=["maturity", "strike", "price", "IV"])
            
            # Construct filename
            filename = f"call_{category}_{std_date}.csv"
            output_path = os.path.join(base_dir, filename)
            
            # Save to CSV
            formatted_df.to_csv(output_path, index=False)
            print(f"Saved {filename} to {base_dir}")
            print(formatted_df.head())
        else:
            print(f"No valid data found for {sheet_name}")
            
    except Exception as e:
        print(f"Error processing sheet {sheet_name}: {e}")

def main():
    """Main function to process all sheets."""
    # Define sheets with consistent naming
    sheet_names = [
        'call_rog_23072024',
        'call_rog_23082024',
        'call_rog_25092024',
        'call_cfr_23072024',
        'call_cfr_23082024',
        'call_cfr_25092024',
        'call_zurn_23072024',
        'call_zurn_23082024'
    ]
    
    # Process each sheet
    for sheet in sheet_names:
        process_and_save(sheet)

if __name__ == "__main__":
    main()

Error processing sheet call_rog_23072024: Worksheet named 'call_rog_23072024' not found
Saved call_rog_23082024.csv to ../data/option_data/23082024
    maturity  strike      price         IV
0 2024-09-20     234  49.574997  33.994099
1 2024-09-20     235  48.574997  33.326797
2 2024-09-20     236  47.574997  32.660213
3 2024-09-20     238  45.599998  31.916555
4 2024-09-20     240  43.624992  31.114557
Saved call_rog_25092024.csv to ../data/option_data/25092024
    maturity  strike      price         IV
0 2024-10-18     210  61.799995  42.994598
1 2024-10-18     215  56.849998  41.726521
2 2024-10-18     220  51.849991  38.028236
3 2024-10-18     225  46.899994  36.034473
4 2024-10-18     230  41.949997  33.622883
Error processing sheet call_cfr_23072024: Worksheet named 'call_cfr_23072024' not found
Saved call_cfr_23082024.csv to ../data/option_data/23082024
    maturity  strike      price         IV
0 2024-09-20     114  22.149994  38.891621
1 2024-09-20     116  20.174995  36.301975